<a href="https://colab.research.google.com/github/SuYouge/colab/blob/master/yolov3_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 单步inference

## 初始化环境

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/yolov3_test"
os.chdir(path)
os.listdir(path)

%cd yolov3-tf2/

# !pip install opencv-python==4.1.1.26 lxml tqdm -e .

%tensorflow_version 2.x
import tensorflow as tf
tf.__version__

## 转换权重文件

In [0]:
!python convert.py

## 初始化网络

In [0]:
import sys
from absl import app, logging, flags
from absl.flags import FLAGS
import time
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs

# flags.DEFINE_stirng 可以设定默认参数候选
flags.DEFINE_string('classes', './data/coco.names', 'path to classes file')
flags.DEFINE_string('weights', './checkpoints/yolov3.tf',
                    'path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('image', './data/girl.png', 'path to input image')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_string('output', './output.jpg', 'path to output image')
flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')

# 初始化程序
app._run_init(['yolov3'], app.parse_flags_with_usage)

# 指定设备
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
physical_devices

## 运行inference

In [0]:
FLAGS.image = 'data/meme.jpg'

logging.info("%s\n"%FLAGS.classes)

logging.info("%s\n"%FLAGS.weights)

if FLAGS.tiny:
    # 声明网络
    yolo = YoloV3Tiny(classes=FLAGS.num_classes)
else:
    yolo = YoloV3(classes=FLAGS.num_classes)

# 加载权重   
yolo.load_weights(FLAGS.weights).expect_partial()
logging.info('weights loaded')

# 加载分类标签
class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
logging.info('classes loaded')

# 读取图片
img_raw = tf.image.decode_image(
    open(FLAGS.image, 'rb').read(), channels=3)

# 扩展图片维度，以便向网络输入
img = tf.expand_dims(img_raw, 0)

# resize以及归一化
img = transform_images(img, FLAGS.size)

t1 = time.time()
# 输入网络，得到输出
boxes, scores, classes, nums = yolo(img)
t2 = time.time()
logging.info('time: {}'.format(t2 - t1))

logging.info('detections:')

# 输出识别结果
for i in range(nums[0]):
    logging.info('\t{}, {}, {}'.format(class_names[int(classes[0][i])],
                                        np.array(scores[0][i]),
                                        np.array(boxes[0][i])))

# 转换图片通道顺序并绘制box
img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

# 显示图片
from IPython.display import Image, display
display(Image(data=bytes(cv2.imencode('.jpg', img)[1]), width=800))